-This note book is the iterartion 02 of the HPCC_GNN_Project-

1.Struct.unpack  is used to strip the headers, this can be done by extracting the pixel data present in the IDTA chunk of the png image.

2.A manual function is written therfore ensures a higher control over the data

In [16]:
import io
from PIL import Image
import binascii
import numpy as np

from io import BytesIO


Since the blob spraying returns the hex data of an image , we are trying to simulate the same by using io and Pil library. (In this note book we have used a png format)

In [ ]:
# img=Image.open("C:/Users/rohan/Pictures/Camera Roll/X13_wallpaper_final_16x9_FHD.jpg")
img=Image.open("C:/Users/rohan/Pictures/Saved Pictures/img01.png")
print(img)


Uing hexlify() imported from binascii library to obtain the hex data of the image.

In [18]:
# with open("C:/Users/rohan/Pictures/Camera Roll/X13_wallpaper_final_16x9_FHD.jpg", "rb") as image_file:
with open("C:/Users/rohan/Pictures/Saved Pictures/img01.png", "rb") as image_file:
    image_hex = binascii.hexlify(image_file.read())
    # print(image_he
    print(image_hex[:1500])
    

b'89504e470d0a1a0a0000000d49484452000007d0000004b0080300000094fa218c0000000467414d410000b18f0bfc6105000000017352474200aece1ce900000300504c54450000002813183b3340332934260e12250c0f271015352d3a03020201010138303d240a0d0706070703043d3541362e3b23080b050405240d113f384448414d332a363a323e090506250f14352e3a3e3743372f3c3227332a161c433c480d07093226314d47520c0b0c1f0f140d0507312530504a54362a3413080b0a090a342c38110f11423c4710090c413a46534d5798979a130b0f190d101007081a10159695984f485467636b26121747404b382e383428320f0d0f2d1b22726e75443e49838084787379888689908e9149434e463e4a17090c2207091d0c106f6b736460684b444f30242e59545d2d20288e8c8f2d1d25160e129e9d9f1d1419868387413a44241f29372c36514b5621171c94929494939779767a8a888b2b181f2d222c13121326161c605b645e596257525b2e28335d5860302a368c8a8d625d657572794b46503c313a17131a453a4222121856505a1615162221226a666d919094231a2069666a5b565f1b090c74707728222d2a25302819203f353f2c26326c6770210c10191819a2a2a3544f587d797e9c9b9e3d333d271d241f1e1f8381871e1922201b24301e24221d2727212b

The hex data into binary data using unhexlify() function.

In [19]:
import binascii
import struct
binary_data = binascii.unhexlify(image_hex)
#
signature = binary_data[0:8]
if signature != b'\x89PNG\r\n\x1a\n':
    raise ValueError("Invalid PNG data")



Search for the IHDR chunk


In [5]:
index = 8
while index < len(binary_data):
    # Extract the chunk length,
    #  After the chunk length is extracted using struct.unpack, the chunk type is extracted using slicing: chunk_type = binary_data[index+4:index+8].
    chunk_length, = struct.unpack("!I", binary_data[index:index+4])
    chunk_type = binary_data[index+4:index+8]
    if chunk_type == b'IHDR':
        break
    else:
        # Skip over this chunk
        index += chunk_length + 12
print(chunk_length,"\t",chunk_type,"\t",index)



13 	 b'IHDR' 	 8


Extract the image dimensions from the IHDR chunk.


In [20]:
width, height = struct.unpack("!II", binary_data[index+8:index+16])
print(width,"\t",height)


2027607194 	 3213794376


Using the struct data to strip off headers

Once the chunk type is found to be IDAT, we can extract the actual image data from the binary data. This is done by slicing the binary data from the position index + 8 to the position index + 8 + chunk_length. The reason we start from index + 8 is because the PNG chunk structure consists of:

4 bytes for the chunk length
4 bytes for the chunk type
chunk_length bytes for the chunk data
4 bytes for the chunk crc   
So the actual image data starts at the position index + 8, which is 4 bytes after the chunk length and 4 bytes after the chunk type. The end position is index + 8 + chunk_length, which is chunk_length bytes after the start position.

In [ ]:
# loop through the dat till we find the IDAT chunk (a chunk where the actual pixel image data start)
while index < len(binary_data):
    chunk_length, = struct.unpack('!I', binary_data[index:index+4])
    chunk_type = binary_data[index+4:index+8]
    if chunk_type == b'IDAT':
        # Found the IDAT chunk
        image_data = binary_data[index+8:index+8+chunk_length]
        break
    index += chunk_length + 12
print(image_data)

As we can see the data has been stripped bcz thereis an error thrown that it cant identify the image file

In [8]:
def strip_headers(image_data):
    with Image.open(BytesIO(image_data)) as im:
        return np.array(im)
stripped_image_array1 = strip_headers(image_data)
image = np.frombuffer(image_data, np.uint8)
print(image)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x00000222F82AC950>